In [1]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import json
import pickle

In [2]:
page = requests.get('https://www.newtral.es/zona-verificacion/fakes/')

In [3]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [4]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [5]:
def get_urls(noticias):
    
    page = requests.get('https://www.newtral.es/wp-json/wp/v2/posts?per_page='+ str(noticias) +'&offset=0&categories=59&_locale=user')

    for e in page.json():
        enlace = e['link']
        lista_enlaces.append(enlace)
        
    return lista_enlaces

In [6]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="entry-title c-article__title").get_text()
    #print(title)
    
    date = soup3.find("time",class_="entry-date c-article__date published updated").get_text().strip('\n')
    #print(date)
    
    tags = soup3.find("ul", class_="tags-links c-pills uk-visible@m")
    tags2 = tags.find_all("a", class_="c-pills__item")
    
    tag_list = []
    
    for t in tags2:
        tag_list.append(t.get_text())
    
    content_list = []
    body = soup3.find("div", class_="entry-content")
    body2 = body.find_all("p", class_="")
    
    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " "))
    
    content = ''.join(content_list[:-1])
    
    article = {"Title": title, "Tags": tag_list, "Publication_date": date, "Content": content, "URL": url, "Fuente": "Newtral.es"}
    
    return article


In [7]:
noticias = 13 #En esta web tenemos 13 noticias de portada y luego se van sumando de 12 en 12 (Podemos seleccionar un número intermedio)
lista_enlaces = []

lista_enlaces = get_urls(noticias)

#print(len(lista_enlaces))
lista_enlaces

lista_articulos = []   
for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)


In [8]:
print(len(lista_enlaces))
print(lista_enlaces)

1
['https://www.newtral.es/policia-asesinato-george-floyd-actor-bulo-gran-montaje/20200605/']


In [9]:
print(lista_articulos)
#print(lista_articulos[0]['Tags'][0])

[{'Title': 'El policía acusado del asesinato de George Floyd no es un actor (y varios desmentidos más de la teoría sobre “el gran montaje”)', 'Tags': ['Estados Unidos', 'fakes', 'George Floyd', 'Violencia'], 'Publication_date': '05 junio 2020', 'Content': 'En los últimos días nos ha llegado a través de nuestro servicio de verificación de Whatsapp (+34 682 58 96 64) de Newtral.es preguntas sobre la veracidad de diversas afirmaciones de una teoría de la conspiración sobre el supuesto asesinato de George Floyd a manos de la policía de Minneapolis. En dicha teoría se vierten diversas afirmaciones como, por ejemplo, que Derek Chauvin, el agente que mantuvo la rodilla sobre el cuello de Floyd y principal acusado, es un actor y que todo formaría parte de un “gran montaje”. Vamos a ir desmontando las afirmaciones objetivamente falsas que hemos recibido.No, no lo es. El detenido es Derek Chauvin como informan infinidad de medios de comunicación. Al ya ex-policía lo detuvieron el 29 de mayo.Poco

In [10]:
from datetime import date

date = date.today().strftime("%d-%m-%Y")

filename = 'Newtral_FN_' + date
print(filename)

outfile = open(filename,'wb')

pickle.dump(lista_articulos, outfile)
outfile.close()

Newtral_FN_07-06-2020


In [11]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': 'El policía acusado del asesinato de George Floyd no es un actor (y varios desmentidos más de la teoría sobre “el gran montaje”)',
  'Tags': ['Estados Unidos', 'fakes', 'George Floyd', 'Violencia'],
  'Publication_date': '05 junio 2020',
  'Content': 'En los últimos días nos ha llegado a través de nuestro servicio de verificación de Whatsapp (+34 682 58 96 64) de Newtral.es preguntas sobre la veracidad de diversas afirmaciones de una teoría de la conspiración sobre el supuesto asesinato de George Floyd a manos de la policía de Minneapolis. En dicha teoría se vierten diversas afirmaciones como, por ejemplo, que Derek Chauvin, el agente que mantuvo la rodilla sobre el cuello de Floyd y principal acusado, es un actor y que todo formaría parte de un “gran montaje”. Vamos a ir desmontando las afirmaciones objetivamente falsas que hemos recibido.No, no lo es. El detenido es Derek Chauvin como informan infinidad de medios de comunicación. Al ya ex-policía lo detuvieron el 29 de may